In [1]:
# 导入必要的库
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

# 数据路径
data_path = r"D:\data\data03.csv"  # 数据文件路径
output_pdf_path = r"D:\data\anova_results01.pdf"  # 输出 PDF 文件路径

# 尝试读取数据，指定不同的编码格式
try:
    # 尝试使用 UTF-8 编码
    data = pd.read_csv(data_path, encoding="utf-8")
    print("使用 UTF-8 编码加载成功！")
except UnicodeDecodeError:
    try:
        # 尝试使用 GBK 编码
        data = pd.read_csv(data_path, encoding="gbk")
        print("使用 GBK 编码加载成功！")
    except UnicodeDecodeError:
        try:
            # 尝试使用 ISO-8859-1 编码
            data = pd.read_csv(data_path, encoding="iso-8859-1")
            print("使用 ISO-8859-1 编码加载成功！")
        except UnicodeDecodeError:
            print("无法确定文件编码，请检查文件格式或手动指定编码。")
            data = None

# 如果数据加载成功，继续后续操作
if data is not None:
    # 查看数据的前几行，确认数据加载正确
    print("\n数据的前几行：")
    print(data.head())

    # 清理列名：将特殊字符替换为下划线，并移除空格
    data.columns = data.columns.str.replace('[ /%+-]', '_', regex=True).str.strip()
    data.columns = data.columns.str.replace('[°Ω]', 'X', regex=True)  # 替换特殊字符为合法字符
    data.columns = data.columns.str.replace(r'^\d+', 'Var_', regex=True)  # 以数字开头的列名前加 "Var_"

    # 确定控制变量和响应变量的列名
    control_columns = data.columns[:11]  # 前11列是控制变量
    response_columns = data.columns[11:]  # 后9列是响应变量

    print("\n清理后的控制变量列名：", control_columns)
    print("清理后的响应变量列名：", response_columns)

    # 初始化 F 值和 P 值矩阵
    f_matrix = pd.DataFrame(index=control_columns, columns=response_columns, dtype=float)
    p_matrix = pd.DataFrame(index=control_columns, columns=response_columns, dtype=float)

    # 对每个控制变量和每个响应变量进行方差分析
    for control_var in control_columns:
        for response_var in response_columns:
            # 修复公式字符串，不使用反引号
            formula = f"{response_var} ~ C({control_var})"  
            try:
                model = ols(formula, data=data).fit()
                anova_results = sm.stats.anova_lm(model, typ=2)
                f_value = anova_results['F'][0]
                p_value = anova_results['PR(>F)'][0]

                # 填充 F 值和 P 值矩阵
                f_matrix.loc[control_var, response_var] = f_value
                p_matrix.loc[control_var, response_var] = p_value

            except Exception as e:
                print(f"对 {control_var} 和 {response_var} 进行方差分析时出错：{e}")

    # 创建 PDF 文件
    with PdfPages(output_pdf_path) as pdf:
        # 绘制 F 值热力图
        plt.figure(figsize=(16, 12))  # 调整图大小以适应特征名
        sns.heatmap(f_matrix, annot=True, fmt=".2f", cmap="viridis", linewidths=0.5, cbar_kws={"label": "F Value"})
        plt.title("ANOVA F Values Heatmap", fontsize=18)
        plt.xlabel("Response Variables", fontsize=16)
        plt.ylabel("Control Variables", fontsize=16)

        # 调整特征名显示
        plt.xticks(rotation=45, ha="right", rotation_mode="anchor")  # 旋转X轴标签
        plt.yticks(rotation=0)  # 保持Y轴标签水平
        plt.tight_layout()  # 自动调整子图参数，以确保子图之间有足够空间
        pdf.savefig()  # 保存到 PDF
        plt.close()

        # 绘制 P 值热力图
        plt.figure(figsize=(16, 12))  # 调整图大小以适应特征名
        sns.heatmap(p_matrix, annot=True, fmt=".2e", cmap="viridis", linewidths=0.5, cbar_kws={"label": "P Value"})
        plt.title("ANOVA P Values Heatmap", fontsize=18)
        plt.xlabel("Response Variables", fontsize=16)
        plt.ylabel("Control Variables", fontsize=16)

        # 调整特征名显示
        plt.xticks(rotation=45, ha="right", rotation_mode="anchor")  # 旋转X轴标签
        plt.yticks(rotation=0)  # 保持Y轴标签水平
        plt.tight_layout()  # 自动调整子图参数，以确保子图之间有足够空间
        pdf.savefig()  # 保存到 PDF
        plt.close()

    print(f"\n所有分析结果已保存到 PDF 文件中：{output_pdf_path}")

使用 UTF-8 编码加载成功！

数据的前几行：
    SBET  Average_pore_size  Total_pore_volume         ID         IG  ID_IG  \
0   8.90           1.836667           5.166667  1352.0000  1566.0000  0.870   
1   6.20           2.866667           6.100000  1352.0000  1566.0000  0.920   
2   5.30           2.816667           4.483333  1352.0000  1566.0000  0.930   
3   0.05           0.210000           4.800000     0.7376     1.0682  0.691   
4  22.00           2.600000           5.000000     0.0711     0.8628  0.082   

          2θ      d002     La    Lc     La_Lc  first_discharge  first_charge  \
0  26.524000  3.355800  100.0  55.6  1.798561       229.537331        327.77   
1  26.506000  3.360000   82.6  44.8  1.843750       249.795936        350.64   
2  26.508000  3.350700   67.0  39.4  1.700508       230.086992        321.71   
3  20.942400  0.934868   55.0  12.1  4.545455       219.073877        260.00   
4  10.969805  1.616398  467.0  53.7  8.696462       209.164796        250.00   

         ICE  Rete